In [ ]:
### Differential Phase Shift QKD Protocol, with 10% Bit Flip Noise ###
### Implementation by Ajanta Das, 27-07-2023 ###

from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, BasicAer
import matplotlib.pyplot as plt
import numpy as np
import random

#Options & Noise goes here - Don't change options variable name & block
options = { 
}
#####Write your code after this line######
# The Circuit

### Alice Setup
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')
qc = QuantumCircuit(q, c)

qc.x(q[0])
qc.cu1(1.91, q[0], q[1])
qc.cx(q[1], q[0])
qc.ch(q[1], q[2])
qc.cx(q[2], q[1])
qc.draw()

qc.barrier()
n=3
phases=[0, 1, 2]
phase_angle=[]
for i in range(n):
  if(phases[i]==0):
    phase_angle.append(0)
  else:
    phase_angle.append(180)
#print(phase_angle)

#qc.z(q[1]) # phase pi
#qc.z(q[2]) # phase pi

qc.s(q[1]) # phase pi/2 added
qc.s(q[1]) # + phase pi/2 again added to set pi phase
qc.s(q[2])
qc.s(q[2])
qc.barrier()

qc.h(q[3])
qc.h(q[4])
qc.cx(q[3], q[0])
qc.cx(q[3], q[1])
qc.h(q[3])
qc.cx(q[4], q[1])
qc.cx(q[4], q[2])
qc.h(q[4])
#qc.barrier()#circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
qc.measure(q[3], c[3],basis='Ensemble',add_param="Z")
qc.measure(q[4], c[4],basis='Ensemble',add_param="Z"))
#qc.draw()

qc.measure(q,c,basis='Ensemble')
backend = BasicAer.get_backend('dm_simulator')

# Noise parameters
options = {}
options1 = {
    #"thermal_factor": 0.2,
    #"decoherence_factor": 1.,
   # "depolarization_factor": 0.5,
    # "bell_depolarization_factor": 0.99,
    #"decay_factor": 1.,
    "rotation_error": {'rx':[0.1, 0.0], 'ry':[0.1, 0.0], 'rz': [0.1, 0.0]}, #10% bit flip noise applied
    # "tsp_model_error": [1., 0.],
    "plot": False
}

# Execution with and without noise
run = execute(qc,backend,**options)
result = run.result()
run_error = execute(qc,backend,**options1)
result_error = run_error.result()
# Final state (probabilities)
prob = result.results[0].data.ensemble_probability
prob1 = result_error.results[0].data.ensemble_probability

labels = prob1.keys()
without_noise = prob.values()
with_noise = prob1.values()

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, without_noise, width, label='Without Noise')
rects2 = ax.bar(x + width/2, with_noise, width, label='With Noise')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Probability')
# ax.set_title('Ensemble Probabilities with Noise')
ax.set_xlabel('States')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
import sys;plt.savefig(sys.stdout.buffer,format="svg")

print(prob) # probability count for without noise
print(prob1) # probability count for with noise
